In [1]:
from datasets.get_datasets import *
from revision import *
from boostsrl import boostsrl
import random

In [14]:
target = 'advisedby'
test_size = 0.2

In [6]:
[facts, pos, neg] = get_uwcse_dataset(target, acceptedPredicates=[
'professor',
'student',
'advisedby',
'tempadvisedby',
'ta',
'hasposition',
'publication',
'inphase',
'courselevel',
'yearsinprogram',
'projectmember',
])

In [7]:
bk = ['professor(+person).',
'student(+person).',
'advisedby(+person,+person).',
'advisedby(+person,-person).',
'advisedby(-person,+person).',
'tempadvisedby(+person,+person).',
'tempadvisedby(+person,-person).',
'tempadvisedby(-person,+person).',
'ta(+course,+person,+quarter).',
'ta(-course,+person,+quarter).',
'ta(+course,-person,+quarter).',
'ta(+course,+person,-quarter).',
'ta(-course,+person,-quarter).',
'ta(+course,-person,-quarter).',
'hasposition(+person,+faculty).',
'hasposition(+person,-faculty).',
'hasposition(-person,+faculty).',
'publication(+title,+person).',
'publication(+title,-person).',
'publication(-title,+person).',
'inphase(+person,+prequals).',
'inphase(+person,-prequals).',
'inphase(-person,+prequals).',
'courselevel(+course,+level).',
'courselevel(+course,-level).',
'courselevel(-course,+level).',
'yearsinprogram(+person,+year).',
'yearsinprogram(-person,+year).',
'yearsinprogram(+person,-year).',
'projectmember(+project,+person).',
'projectmember(+project,-person).',
'projectmember(-project,+person).']
#'sameproject(project, project).',
#'samecourse(course, course).',
#'sameperson(person, person).',]

background = boostsrl.modes(bk, [target], useStdLogicVariables=False, treeDepth=8, nodeSize=3, numOfClauses=8)

In [15]:
# shuffle all examples
random.shuffle(pos)
random.shuffle(neg)
neg = neg[:len(pos)]

# separate train and test
test_pos = pos[:int(test_size*len(pos))]
test_neg = neg[:int(test_size*len(neg))]
train_pos = pos[int(test_size*len(pos)):]
train_neg = neg[int(test_size*len(neg)):]

print('Train pos size: %s' % len(train_pos))
print('Train neg size: %s' % len(train_neg))
print('Test pos size: %s' % len(test_pos))
print('Test neg size: %s' % len(test_neg))

Train pos size: 91
Train neg size: 91
Test pos size: 22
Test neg size: 22


## Learning 10 boosted trees and performing inference with 10 boosted trees

In [18]:
delete_model_files()
model = boostsrl.train(background, train_pos, train_neg, facts, trees=10)
learning_time = model.traintime()
will = model.get_will_produced_tree()
structured = model.get_structured_tree().copy()
# if it is using boosted trees
# do inference with the combined one
#if trees > 1:
#    os.rename('boostsrl/train/models/bRDNs/Trees/'+ target +'Tree0.tree', 'boostsrl/train/models/bRDNs/Trees/'+ target +'Tree0_temp.tree')
#    os.rename('boostsrl/train/models/bRDNs/Trees/CombinedTreesTreeFile'+ target +'.tree', 'boostsrl/train/models/bRDNs/Trees/'+ target +'Tree0.tree')
results = boostsrl.test(model, test_pos, test_neg, facts, trees=10)
inference_time = results.testtime()
t_results = results.summarize_results()
print('WILL-Produced Tree:')
print_will_produced_tree(will)
print('\n')
print('Results:')
print(t_results)
print('\n')
print('Total learning time: %s seconds' % learning_time)
print('Total inference time: %s seconds' % inference_time)
print('AUC ROC: %s' % t_results['AUC ROC'])

WILL-Produced Tree:
% FOR advisedby(A, B):
%   if ( professor(B), student(A) )
%   then if ( publication(C, A), publication(C, B) )
%   | then return 3.547568576368876;  // std dev = 2,167, 380,000 (wgt'ed) examples reached here.  /* #neg=20 #pos=360 */
%   | else if ( publication(D, A), publication(E, B) )
%   | | then if ( tempadvisedby(F, B), ta(G, F, H) )
%   | | | then return -1.6414597806436932;  // std dev = 0,696, 30,000 (wgt'ed) examples reached here.  /* #neg=30 */
%   | | | else if ( ta(I, A, J) )
%   | | | | then return 4.33846043041414;  // std dev = 0,249, 40,000 (wgt'ed) examples reached here.  /* #pos=40 */
%   | | | | else if ( tempadvisedby(K, B) )
%   | | | | | then return -1.3018384146137594;  // std dev = 0,327, 20,000 (wgt'ed) examples reached here.  /* #neg=20 */
%   | | | | | else return 1.1908147955791861;  // std dev = 42,077, 80,000 (wgt'ed) examples reached here.  /* #neg=50 #pos=30 */
%   | | else if ( ta(L, A, M) )
%   | | | then return 3.651362347439959; 

## Performing inference with the Combined tree from the previous 10 boosted trees

In [19]:
# if it is using boosted trees
# do inference with the combined one
os.rename('boostsrl/train/models/bRDNs/Trees/'+ target +'Tree0.tree', 'boostsrl/train/models/bRDNs/Trees/'+ target +'Tree0_temp.tree')
os.rename('boostsrl/train/models/bRDNs/Trees/CombinedTreesTreeFile'+ target +'.tree', 'boostsrl/train/models/bRDNs/Trees/'+ target +'Tree0.tree')
results = boostsrl.test(model, test_pos, test_neg, facts, trees=1)
inference_time = results.testtime()
t_results = results.summarize_results()
print('WILL-Produced Tree:')
print_will_produced_tree(will)
print('\n')
print('Results:')
print(t_results)
print('\n')
print('Total learning time: %s seconds' % learning_time)
print('Total inference time: %s seconds' % inference_time)
print('AUC ROC: %s' % t_results['AUC ROC'])

WILL-Produced Tree:
% FOR advisedby(A, B):
%   if ( professor(B), student(A) )
%   then if ( publication(C, A), publication(C, B) )
%   | then return 3.547568576368876;  // std dev = 2,167, 380,000 (wgt'ed) examples reached here.  /* #neg=20 #pos=360 */
%   | else if ( publication(D, A), publication(E, B) )
%   | | then if ( tempadvisedby(F, B), ta(G, F, H) )
%   | | | then return -1.6414597806436932;  // std dev = 0,696, 30,000 (wgt'ed) examples reached here.  /* #neg=30 */
%   | | | else if ( ta(I, A, J) )
%   | | | | then return 4.33846043041414;  // std dev = 0,249, 40,000 (wgt'ed) examples reached here.  /* #pos=40 */
%   | | | | else if ( tempadvisedby(K, B) )
%   | | | | | then return -1.3018384146137594;  // std dev = 0,327, 20,000 (wgt'ed) examples reached here.  /* #neg=20 */
%   | | | | | else return 1.1908147955791861;  // std dev = 42,077, 80,000 (wgt'ed) examples reached here.  /* #neg=50 #pos=30 */
%   | | else if ( ta(L, A, M) )
%   | | | then return 3.651362347439959; 

## Performing parameter learning from previous Combined Tree

In [21]:
delete_model_files()
model = boostsrl.train(background, train_pos, train_neg, facts, refine=get_refine_file(structured), trees=1)
learning_time = model.traintime()
will = model.get_will_produced_tree()
structured = model.get_structured_tree().copy()
# if it is using boosted trees
# do inference with the combined one
#if trees > 1:
#    os.rename('boostsrl/train/models/bRDNs/Trees/'+ target +'Tree0.tree', 'boostsrl/train/models/bRDNs/Trees/'+ target +'Tree0_temp.tree')
#    os.rename('boostsrl/train/models/bRDNs/Trees/CombinedTreesTreeFile'+ target +'.tree', 'boostsrl/train/models/bRDNs/Trees/'+ target +'Tree0.tree')
results = boostsrl.test(model, test_pos, test_neg, facts, trees=1)
inference_time = results.testtime()
t_results = results.summarize_results()
print('WILL-Produced Tree:')
print_will_produced_tree(will)
print('\n')
print('Results:')
print(t_results)
print('\n')
print('Total learning time: %s seconds' % learning_time)
print('Total inference time: %s seconds' % inference_time)
print('AUC ROC: %s' % t_results['AUC ROC'])

WILL-Produced Tree:
% FOR advisedby(A, B):
%   if ( professor(B), student(A) )
%   then if ( publication(C, A), publication(C, B) )
%   | then return 0.805517356152143;  // std dev = 1,376, 38,000 (wgt'ed) examples reached here.  /* #neg=2 #pos=36 */
%   | else if ( publication(D, A), publication(E, B) )
%   | | then if ( tempadvisedby(F, B), ta(G, F, H) )
%   | | | then return -0.14185106490048777;  // std dev = 0,000, 3,000 (wgt'ed) examples reached here.  /* #neg=3 */
%   | | | else if ( ta(I, A, J) )
%   | | | | then return 0.8581489350995122;  // std dev = 0,000, 4,000 (wgt'ed) examples reached here.  /* #pos=4 */
%   | | | | else if ( tempadvisedby(K, B) )
%   | | | | | then return -0.14185106490048777;  // std dev = 0,000, 2,000 (wgt'ed) examples reached here.  /* #neg=2 */
%   | | | | | else return 0.23314893509951226;  // std dev = 1,369, 8,000 (wgt'ed) examples reached here.  /* #neg=5 #pos=3 */
%   | | else if ( ta(L, A, M) )
%   | | | then return 0.8126943896449667;  // std

## Experiment (10 runs of 10-fold cross-validation)

In [35]:
from sklearn.model_selection import KFold
import numpy as np
import sys
import time

inference_10boosted = []
inference_combined = []
inference_pl = []

start = time.time()
[facts, positives, negatives] = get_uwcse_dataset(target, acceptedPredicates=[
'professor',
'student',
'advisedby',
'tempadvisedby',
'ta',
'hasposition',
'publication',
'inphase',
'courselevel',
'yearsinprogram',
'projectmember',
])

pos = list(positives)
neg = list(negatives)

for run in range(1,11):
    # shuffle all examples
    random.shuffle(pos)
    random.shuffle(neg)
    neg = neg[:len(pos)]

    pos = np.array(pos)
    neg = np.array(neg)

    fold = 0
    kf = KFold(n_splits=10)
    for train_index, test_index in kf.split(pos):
        train_pos, test_pos = pos[train_index], pos[test_index]
        train_neg, test_neg = neg[train_index], neg[test_index]
        fold += 1

        # learning 10 boosted trees
        delete_model_files()
        model = boostsrl.train(background, train_pos, train_neg, facts, trees=10)
        structured = model.get_structured_tree().copy()
        results = boostsrl.test(model, test_pos, test_neg, facts, trees=10)
        t_results = results.summarize_results()
        inference_10boosted.append(t_results['AUC ROC'])

        # do inference with the combined tree
        os.rename('boostsrl/train/models/bRDNs/Trees/'+ target +'Tree0.tree', 'boostsrl/train/models/bRDNs/Trees/'+ target +'Tree0_temp.tree')
        os.rename('boostsrl/train/models/bRDNs/Trees/CombinedTreesTreeFile'+ target +'.tree', 'boostsrl/train/models/bRDNs/Trees/'+ target +'Tree0.tree')
        results = boostsrl.test(model, test_pos, test_neg, facts, trees=1)
        t_results = results.summarize_results()
        inference_combined.append(t_results['AUC ROC'])

        # parameter learning
        delete_model_files()
        model = boostsrl.train(background, train_pos, train_neg, facts, refine=get_refine_file(structured), trees=1)
        results = boostsrl.test(model, test_pos, test_neg, facts, trees=1)
        t_results = results.summarize_results()
        inference_pl.append(t_results['AUC ROC'])

        sys.stdout.write('\rRun: %s, Fold: %s, 10_boosted_trees: %s, combined: %s, pl: %s, time: %s' % (run, fold, inference_10boosted[-1], inference_combined[-1], inference_pl[-1], time.strftime('%H:%M:%S', time.gmtime(time.time()-start))))
        sys.stdout.flush()

Run: 10, Fold: 10, 10_boosted_trees: 0.954545, combined: 0.92562, pl: 0.92562, time: 00:42:50

In [42]:
import pandas as pd

inference_10boosted = np.array(inference_10boosted)
inference_combined = np.array(inference_combined)
inference_pl = np.array(inference_pl)

data = [['10 boosted trees', '%s +/- %.4f' % (inference_10boosted.mean(), inference_10boosted.std())],
       ['Combined 10 boosted trees', '%s +/- %.4f' % (inference_combined.mean(), inference_combined.std())],
       ['Parameter learning', '%s +/- %.4f' % (inference_pl.mean(), inference_pl.std())]]
pd.DataFrame(data, columns=['', 'AUC ROC'])

,,AUC ROC
0,10 boosted trees,0.95455213 +/- 0.0456
1,Combined 10 boosted trees,0.9153463299999999 +/- 0.0747
2,Parameter learning,0.91537445 +/- 0.0781


In [40]:
inference_10boosted

array([0.993056, 0.986111, 0.944444, 0.921488, 0.859504, 0.991736,
       0.938017, 0.958678, 1.      , 0.991736, 0.878472, 1.      ,
       1.      , 1.      , 0.975207, 0.909091, 0.88843 , 0.909091,
       0.975207, 0.942149, 0.993056, 0.986111, 0.993056, 1.      ,
       0.942149, 1.      , 0.921488, 0.966942, 0.892562, 0.966942,
       0.923611, 0.989583, 0.909722, 0.933884, 0.966942, 1.      ,
       1.      , 0.917355, 0.793388, 1.      , 0.875   , 0.916667,
       0.9375  , 0.983471, 0.900826, 0.991736, 1.      , 0.991736,
       1.      , 0.942149, 0.951389, 0.857639, 0.951389, 1.      ,
       1.      , 0.834711, 0.966942, 0.85124 , 1.      , 0.942149,
       0.895833, 0.965278, 0.958333, 1.      , 0.954545, 0.987603,
       1.      , 0.966942, 0.88843 , 1.      , 0.916667, 0.986111,
       0.951389, 0.942149, 0.904959, 0.983471, 1.      , 0.938017,
       0.96281 , 1.      , 0.96875 , 1.      , 0.916667, 0.991736,
       0.991736, 0.884298, 0.991736, 1.      , 0.971074, 0.938

In [43]:
inference_combined

array([0.979167, 0.979167, 0.944444, 0.942149, 0.863636, 0.88843 ,
       0.876033, 0.971074, 0.909091, 0.913223, 0.850694, 1.      ,
       1.      , 1.      , 0.991736, 0.859504, 0.743802, 0.913223,
       0.904959, 0.971074, 0.954861, 0.895833, 0.927083, 0.991736,
       0.789256, 0.991736, 0.933884, 0.842975, 0.669421, 0.921488,
       0.861111, 0.899306, 0.798611, 0.958678, 0.975207, 1.      ,
       0.991736, 0.942149, 0.876033, 1.      , 0.829861, 0.951389,
       0.736111, 0.971074, 0.801653, 0.917355, 1.      , 1.      ,
       0.909091, 0.950413, 0.909722, 0.954861, 0.881944, 1.      ,
       1.      , 0.867769, 0.913223, 0.880165, 0.917355, 0.96281 ,
       0.784722, 0.895833, 0.979167, 0.909091, 0.979339, 0.966942,
       1.      , 0.760331, 0.933884, 1.      , 0.90625 , 0.975694,
       0.833333, 0.917355, 0.772727, 0.946281, 0.909091, 0.933884,
       0.880165, 0.921488, 0.9375  , 1.      , 0.927083, 0.900826,
       0.979339, 0.863636, 0.834711, 0.909091, 0.884298, 0.863

In [44]:
inference_pl

array([0.979167, 0.979167, 0.944444, 0.942149, 0.863636, 0.904959,
       0.876033, 0.971074, 0.909091, 0.913223, 0.847222, 1.      ,
       1.      , 1.      , 0.991736, 0.859504, 0.743802, 0.913223,
       0.904959, 0.971074, 0.954861, 0.895833, 0.927083, 0.991736,
       0.789256, 0.991736, 0.933884, 0.826446, 0.669421, 0.921488,
       0.861111, 0.899306, 0.798611, 0.958678, 0.975207, 1.      ,
       1.      , 0.942149, 0.710744, 1.      , 0.829861, 0.958333,
       0.736111, 0.971074, 0.797521, 0.913223, 1.      , 1.      ,
       0.909091, 0.950413, 0.909722, 0.954861, 0.881944, 1.      ,
       1.      , 0.867769, 0.979339, 0.880165, 0.913223, 0.96281 ,
       0.788194, 0.895833, 0.979167, 0.909091, 0.971074, 0.966942,
       1.      , 0.760331, 0.933884, 1.      , 0.90625 , 0.975694,
       0.833333, 0.917355, 0.772727, 0.946281, 0.909091, 0.933884,
       0.880165, 0.96281 , 0.9375  , 1.      , 0.927083, 0.904959,
       0.979339, 0.863636, 0.830579, 0.909091, 0.950413, 0.863